# Construiește un model de regresie: pregătește și vizualizează datele

## **Regresie liniară pentru dovleci - Lecția 2**
#### Introducere

Acum că ai configurat instrumentele necesare pentru a începe să construiești modele de învățare automată folosind Tidymodels și Tidyverse, ești pregătit să începi să pui întrebări despre datele tale. Pe măsură ce lucrezi cu date și aplici soluții de învățare automată, este foarte important să înțelegi cum să formulezi întrebarea corectă pentru a valorifica în mod adecvat potențialul setului tău de date.

În această lecție, vei învăța:

- Cum să pregătești datele pentru construirea unui model.

- Cum să folosești `ggplot2` pentru vizualizarea datelor.

Întrebarea la care vrei să obții un răspuns va determina ce tip de algoritmi de învățare automată vei utiliza. Iar calitatea răspunsului pe care îl primești va depinde în mare măsură de natura datelor tale.

Să vedem acest lucru printr-un exercițiu practic.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Ilustrație de @allison_horst</figcaption>


<!--![Ilustrație de \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Ilustrație de \@allison_horst-->


## 1. Importarea datelor despre dovleci și invocarea Tidyverse

Vom avea nevoie de următoarele pachete pentru a explora această lecție:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) este o [colecție de pachete R](https://www.tidyverse.org/packages) concepută pentru a face știința datelor mai rapidă, mai ușoară și mai distractivă!

Le poți instala astfel:

`install.packages(c("tidyverse"))`

Scriptul de mai jos verifică dacă ai pachetele necesare pentru a finaliza acest modul și le instalează pentru tine în cazul în care lipsesc unele.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Acum, să pornim câteva pachete și să încărcăm [datele](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) furnizate pentru această lecție!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

O privire rapidă cu `glimpse()` arată imediat că există valori lipsă și un amestec de șiruri de caractere (`chr`) și date numerice (`dbl`). Coloana `Date` este de tip caracter, iar există și o coloană ciudată numită `Package`, unde datele sunt un amestec de `sacks`, `bins` și alte valori. De fapt, datele sunt un pic dezordonate 😤.

De altfel, nu este foarte comun să primești un set de date complet pregătit pentru a crea un model de ML direct din cutie. Dar nu-ți face griji, în această lecție vei învăța cum să pregătești un set de date brut folosind biblioteci standard din R 🧑‍🔧. De asemenea, vei învăța diverse tehnici pentru a vizualiza datele. 📈📊
<br>

> O recapitulare: Operatorul pipe (`%>%`) efectuează operațiuni într-o succesiune logică, transmițând un obiect mai departe într-o funcție sau expresie. Poți considera operatorul pipe ca și cum ai spune "și apoi" în codul tău.


## 2. Verificarea datelor lipsă

Una dintre cele mai frecvente probleme cu care se confruntă oamenii de știință în domeniul datelor este existența datelor incomplete sau lipsă. R reprezintă valorile lipsă sau necunoscute printr-o valoare specială: `NA` (Not Available).

Deci, cum putem afla dacă cadrul de date conține valori lipsă?
<br>
-   O metodă directă ar fi utilizarea funcției de bază din R `anyNA`, care returnează obiecte logice `TRUE` sau `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Grozav, se pare că lipsesc unele date! Acesta este un punct bun de plecare.

-   O altă metodă ar fi să folosești funcția `is.na()` care indică ce elemente individuale din coloană lipsesc, returnând un `TRUE` logic.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Bine, am terminat treaba, dar cu un cadru de date atât de mare, ar fi ineficient și practic imposibil să revizuim toate rândurile și coloanele individual 😴.

-   O modalitate mai intuitivă ar fi să calculăm suma valorilor lipsă pentru fiecare coloană:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Mult mai bine! Lipsesc unele date, dar poate că nu va conta pentru sarcina de față. Să vedem ce va aduce analiza ulterioară.

> Pe lângă seturile impresionante de pachete și funcții, R are o documentație foarte bună. De exemplu, folosește `help(colSums)` sau `?colSums` pentru a afla mai multe despre funcție.


## 3. Dplyr: O gramatică a manipulării datelor

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Ilustrație de @allison_horst</figcaption>


<!--![Ilustrație de \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Ilustrație de \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), un pachet din Tidyverse, este o gramatică a manipulării datelor care oferă un set consecvent de verbe ce te ajută să rezolvi cele mai comune provocări legate de manipularea datelor. În această secțiune, vom explora câteva dintre verbele din dplyr!  
<br>


#### dplyr::select()

`select()` este o funcție din pachetul `dplyr` care te ajută să alegi coloanele pe care vrei să le păstrezi sau să le excluzi.

Pentru a face cadrul de date mai ușor de utilizat, elimină mai multe dintre coloanele sale folosind `select()`, păstrând doar coloanele de care ai nevoie.

De exemplu, în acest exercițiu, analiza noastră va implica coloanele `Package`, `Low Price`, `High Price` și `Date`. Hai să selectăm aceste coloane.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` este o funcție din pachetul `dplyr` care te ajută să creezi sau să modifici coloane, păstrând în același timp coloanele existente.

Structura generală a funcției mutate este:

`data %>%   mutate(new_column_name = what_it_contains)`

Să testăm funcția `mutate` folosind coloana `Date` prin realizarea următoarelor operații:

1.  Convertirea datelor (în prezent de tip caracter) într-un format de lună (acestea sunt date din SUA, deci formatul este `MM/DD/YYYY`).

2.  Extrage luna din date într-o coloană nouă.

În R, pachetul [lubridate](https://lubridate.tidyverse.org/) face mai ușoară lucrul cu date și ore. Așadar, să folosim `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` și să vedem cum putem atinge obiectivele de mai sus. Putem elimina coloana Date, deoarece nu o vom mai folosi în operațiile ulterioare.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Ura! 🤩

În continuare, să creăm o nouă coloană `Price`, care reprezintă prețul mediu al unui dovleac. Acum, să calculăm media dintre coloanele `Low Price` și `High Price` pentru a completa noua coloană Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Yeees!💪

„Dar stai puțin!”, vei spune după ce ai răsfoit întregul set de date cu `View(pumpkins)`, „E ceva ciudat aici!”🤔

Dacă te uiți la coloana `Package`, dovlecii sunt vânduți în multe configurații diferite. Unii sunt vânduți în măsuri de `1 1/9 bushel`, alții în măsuri de `1/2 bushel`, unii per dovleac, alții per kilogram, iar alții în cutii mari cu lățimi variabile.

Hai să verificăm asta:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Uimitor!👏

Se pare că dovlecii sunt foarte greu de cântărit în mod constant, așa că hai să-i filtrăm selectând doar dovlecii care conțin șirul *bushel* în coloana `Package` și să punem acest lucru într-un nou cadru de date `new_pumpkins`.


#### dplyr::filter() și stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): creează un subset de date care conține doar **rândurile** ce îndeplinesc condițiile tale, în acest caz, dovleci cu șirul *bushel* în coloana `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): detectează prezența sau absența unui model într-un șir de caractere.

Pachetul [`stringr`](https://github.com/tidyverse/stringr) oferă funcții simple pentru operațiuni comune cu șiruri de caractere.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Puteți observa că am restrâns selecția la aproximativ 415 rânduri de date care conțin dovleci la sac. 🤩  
<br>


#### dplyr::case_when()

**Dar stai! Mai este un lucru de făcut**

Ai observat că cantitatea de bushel variază pe rânduri? Trebuie să normalizezi prețurile astfel încât să afișezi prețul pe bushel, nu pe 1 1/9 sau 1/2 bushel. Este momentul să facem câteva calcule pentru a standardiza acest lucru.

Vom folosi funcția [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) pentru a *modifica* coloana Price în funcție de anumite condiții. `case_when` îți permite să vectorizezi mai multe declarații `if_else()`.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Acum putem analiza prețul pe unitate bazat pe măsurarea în busheli. Tot acest studiu despre busheli de dovleci, însă, arată cât de `important` este să `înțelegi natura datelor tale`!

> ✅ Conform [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), greutatea unui bushel depinde de tipul de produs, deoarece este o măsurătoare de volum. "Un bushel de roșii, de exemplu, ar trebui să cântărească 56 de livre... Frunzele și verdețurile ocupă mai mult spațiu cu o greutate mai mică, așa că un bushel de spanac cântărește doar 20 de livre." Este destul de complicat! Să nu ne batem capul cu o conversie din busheli în livre și, în schimb, să stabilim prețul pe bushel. Tot acest studiu despre busheli de dovleci, însă, arată cât de important este să înțelegi natura datelor tale!
>
> ✅ Ai observat că dovlecii vânduți la jumătate de bushel sunt foarte scumpi? Poți să-ți dai seama de ce? Indiciu: dovlecii mici sunt mult mai scumpi decât cei mari, probabil pentru că sunt mult mai mulți într-un bushel, având în vedere spațiul neutilizat ocupat de un dovleac mare și gol pentru plăcintă.


Acum, în cele din urmă, doar pentru distracție 💁‍♀️, să mutăm și coloana Lună în prima poziție, adică `înainte` de coloana `Pachet`.

`dplyr::relocate()` este utilizat pentru a schimba pozițiile coloanelor.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Bravo! 👌 Acum ai un set de date curat și bine organizat pe care poți construi noul tău model de regresie!  
<br>


## 4. Vizualizarea datelor cu ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografic de Dasani Madipalli</figcaption>


<!--![Infografic de Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Există o zicală *înțeleaptă* care spune astfel:

> "Graficul simplu a adus mai multe informații în mintea analistului de date decât orice alt dispozitiv." --- John Tukey

O parte din rolul unui specialist în știința datelor este să demonstreze calitatea și natura datelor cu care lucrează. Pentru a face acest lucru, ei creează adesea vizualizări interesante, cum ar fi grafice, diagrame și scheme, care arată diferite aspecte ale datelor. În acest fel, pot evidenția vizual relații și lacune care altfel ar fi greu de descoperit.

Vizualizările pot ajuta, de asemenea, la determinarea tehnicii de învățare automată cea mai potrivită pentru datele respective. De exemplu, un grafic de dispersie care pare să urmeze o linie indică faptul că datele sunt un candidat bun pentru un exercițiu de regresie liniară.

R oferă mai multe sisteme pentru crearea graficelor, dar [`ggplot2`](https://ggplot2.tidyverse.org/index.html) este unul dintre cele mai elegante și versatile. `ggplot2` îți permite să compui grafice prin **combinarea componentelor independente**.

Să începem cu un grafic de dispersie simplu pentru coloanele Price și Month.

Așadar, în acest caz, vom începe cu [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), vom furniza un set de date și o mapare estetică (cu [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), apoi vom adăuga straturi (precum [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) pentru grafice de dispersie.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Este un grafic util 🤷? Te surprinde ceva la el?

Nu este deosebit de util, deoarece tot ce face este să afișeze datele tale ca o împrăștiere de puncte într-o anumită lună.
<br>


### **Cum îl facem util?**

Pentru a afișa date utile în grafice, de obicei trebuie să grupăm datele într-un anumit mod. De exemplu, în cazul nostru, calcularea prețului mediu al dovlecilor pentru fiecare lună ar oferi mai multe informații despre tiparele de bază din datele noastre. Acest lucru ne conduce la încă o privire rapidă asupra **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Agregarea grupată în R poate fi calculată ușor folosind

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` schimbă unitatea de analiză de la întregul set de date la grupuri individuale, cum ar fi pe lună.

-   `dplyr::summarize()` creează un nou cadru de date cu o coloană pentru fiecare variabilă de grupare și o coloană pentru fiecare statistică sumară pe care ai specificat-o.

De exemplu, putem folosi `dplyr::group_by() %>% summarize()` pentru a grupa dovlecii în funcție de coloana **Month** și apoi să calculăm **prețul mediu** pentru fiecare lună.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Succint!✨

Caracteristicile categorice, cum ar fi lunile, sunt mai bine reprezentate folosind un grafic cu bare 📊. Straturile responsabile pentru graficele cu bare sunt `geom_bar()` și `geom_col()`. Consultă `?geom_bar` pentru a afla mai multe.

Hai să creăm unul!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Aceasta este o vizualizare a datelor mai utilă! Se pare că indică faptul că prețul cel mai ridicat pentru dovleci apare în septembrie și octombrie. Se potrivește asta cu așteptările tale? De ce sau de ce nu?

Felicitări pentru finalizarea celei de-a doua lecții 👏! Ai pregătit datele pentru construirea modelului, apoi ai descoperit mai multe informații folosind vizualizări!



---

**Declinarea responsabilității**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru a asigura acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa nativă ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
